In [9]:
import os
import sys
from pathlib import Path

import cv2 as cv
import cv2
import h5py
import matplotlib.pyplot as plt
import napari
import numpy as np
import pandas as pd
import skimage
from joblib import Parallel, delayed
from skimage import exposure, io
from skimage.util import img_as_ubyte
from tqdm.notebook import tqdm, trange

In [10]:
p_dir = (Path().cwd().parents[0]).absolute()
data_dir = p_dir / "data"
match_info_dir = data_dir / "match"

In [11]:
%load_ext autoreload
%autoreload 2

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

import utils as my_utils
import transform 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
experiment = "Lung"
cores = ['B5', 'C6', 'D4', 'E4', 'E6', 'F4', 'F7']

# Read SIMS

In [18]:
h5_data_dir = p_dir / "data" / "h5"

def get_imgs_sims(experiment, name):
    with h5py.File(h5_data_dir / f"{experiment}.hdf5", "r") as f:
        imgs = f[name][:]
        labels = list(f[name].attrs["labels"])
    return imgs, labels

# Read mask image
def get_masks(mask_folder, dataset):
    """
    Function to get all mask from mask forlder
    """
    # Read masks
    masks = {}

    for (dirpath, dirnames, filenames) in os.walk(mask_folder):
        for name in sorted(filenames):
            if "tif" in name and dataset in name:
                filename = os.path.join(dirpath, name)
                img = skimage.io.imread(filename)
                condition = name.split(".")[0]
                masks[condition] = img
            else:
                continue
    return masks

In [19]:
for i, core in enumerate(cores):
    datasets = [f'{core}_{i}' for i in range(1,5)]
    masks = get_masks(data_dir / "masks", f"{experiment}_{core}")
    for j, dataset in enumerate(datasets):
        try:
            info = my_utils.load_pkl(match_info_dir / f"{experiment}_{dataset}.pickle")
        except:
            continue
        print(dataset)
        
        # --- Read mask
        mask = masks["_".join([experiment, dataset])]

        # --- Read tof-sims images
        imgs, labels = get_imgs_sims(experiment, dataset)

B5_1
B5_2
B5_3
C6_1
C6_2
C6_3
C6_4
D4_1
D4_2
D4_3
E4_1
E4_2
E4_3
E4_4
E6_1
E6_2
E6_3
F4_1
F4_2
F4_3
F7_1
F7_2
F7_3
F7_4
